In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
import torch
import os
from collections import Counter
import sklearn
import psutil
import numpy as np
import time
from loguru import logger

import pandas as pd
# import scanpy as sc
from tqdm import tqdm
import random
import sys
import multiprocessing
import anndata as ad
import gc
import pickle
# from torch.nn.utils.rnn import pad_sequence

root_path = os.path.abspath('/home/chenyx/scMulan/cellgpt_v1/')
sys.path.append(os.path.abspath(root_path))

# from imp import reload
# reload(hf_tokenizer) # also reload(mymodule)

# from utils.hf_tokenizer import cellGenesisTokenizer

#data_root_path = '/home/bianhaiyang/tmp_dataHub/datasets/'
#dataset = 'ECA_GO'
#dataPath = os.path.join(data_root_path,dataset)
dataPath = "/home/bianhaiyang/tmp_dataHub/datasets/"
assert os.path.exists(dataPath)
tmp_dataPath = os.path.join(dataPath,'tmp_files')

In [ ]:
from natsort import natsorted
import seaborn as sns

In [ ]:
from utils.hf_tokenizer import cellGenesisTokenizer

In [ ]:
from model.model import GPTConfig, cellGPTModel
import torch.nn.functional as F
from model import model_kvcache

In [ ]:
meta_info = torch.load(os.path.join(dataPath,'meta_info.pt'))
chars = meta_info['token_set']
tokenizer = cellGenesisTokenizer(chars)

In [ ]:
n_express_level = 10

In [ ]:
from multiprocessing import Pool
import multiprocessing
from tools.generation.denovoGeneration import generate_denovo

# Init

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

In [ ]:
def generate_prompt_for_cg(idx,meta_data,meta_pool):
    meta_num = torch.randint(0,len(meta_pool),(1,)).item()
    task_cols = meta_pool[meta_num]
    metadata_series = meta_data.loc[idx, task_cols]
    metadata_list = metadata_series.tolist()
    while 'Unclassified' in metadata_list:
        meta_num = torch.randint(0,len(meta_pool),(1,)).item()
        task_cols = meta_pool[meta_num]
        metadata_series = meta_data.loc[idx, task_cols]
        metadata_list = metadata_series.tolist()
    return metadata_list

In [ ]:
def get_binned_expression(row,n_expression_level):
    max_expr = row.max()
    bins = np.linspace(0, max_expr, n_expression_level + 1)
    binned_expr = np.digitize(row, bins, right=True) 
    return pd.Series(binned_expr/n_expression_level, index=row.index)

In [ ]:
def convert_cellsentence_ele_to_df(cellGene_ele, n_expression_level, task = 'cs'):
    dfList = []
    ## fetch gene expression
    for c,e in tqdm(cellGene_ele):
        st = c.index(2207)+1
        c_list = tokenizer.convert_ids_to_tokens(c[st:-1])
        ele = e[st:-1]
        dfDict = {}
        for name,ele in zip(c_list,ele):
            dfDict[name] = ele/n_expression_level
        dfList.append(dfDict)
    gDF = pd.DataFrame(dfList).fillna(0)
    return gDF

### load model

In [ ]:
from model.model import GPTConfig, cellGPTModel
import torch.nn.functional as F
from model.model_kvcache import cellGPTModel_kv

In [ ]:
ckp = torch.load('/nfs/public/cell_gpt_data/dataHub/datasets/datasets/ECA_GO/model_hub/ckpt245000.pt')
# ckp['model_args'].pop('beta')
gptconf = GPTConfig(**ckp['model_args'])
model_kv = cellGPTModel_kv(gptconf)
model_kv = model_kv.cuda()
model_kv.load_state_dict(ckp['model'])
model_kv.eval()

### generation

In [ ]:
def pack_arg(arg,n_process):
    return [arg for _ in range(n_process)]

#### Generation

In [ ]:
def generate_combinations(base_dict, variable1_name, variable2_name, start_value, end_value, step):
    # Initialize the list
    combinations = []
    
    # Generat the combination of different level of 'GJA5' 和 'PLVAP'
    for value1 in range(int(start_value * n_express_level), int((end_value + step) * n_express_level), int(step * n_express_level)):
        for value2 in range(int(start_value * n_express_level), int((end_value + step) * n_express_level), int(step * n_express_level)):
            new_dict = base_dict.copy()
            new_dict[variable1_name] = value1
            new_dict[variable2_name] = value2
            combinations.append(new_dict)
    
    return combinations
    
def repeat_combinations(combinations, n):
    repeated_list = []
    for _ in range(n):
        repeated_list.extend(combinations)
    return repeated_list


In [ ]:
base_dict = {'Heart': 0, 'Vascular endothelial cell': 0, '<SPToken1>': 0}

# select genes
variable1_name = 'GJA5'
variable2_name = 'PLVAP'

# set range and step_length
start_value = 0.0
end_value = 1
step = 0.1

combinations = generate_combinations(base_dict, variable1_name, variable2_name, start_value, end_value, step)
combinations.append(base_dict)

repeat_n = 200
repeated_combinations = repeat_combinations(combinations, repeat_n)

In [ ]:
if __name__ == '__main__': 
    multiprocessing.set_start_method('spawn', force=True)
    print('当前母进程: {}'.format(os.getpid()))
    start = time.time()
    gamma = 0.1 
    use_gpu_device = [0, 1, 2, 3] 
    n_process_in_each_gpu = 3 
    ckp_path = '/nfs/public/cell_gpt_data/modelHub/full_model/ckpt135000.pt' 
    results_dir = f'/nfs/public/cell_gpt_data/dataHub/generated_cells/VEC_Gradient_FullLength/'
    world_size = len(use_gpu_device)
    n_process = n_process_in_each_gpu * world_size
    process_idx = list(range(n_process))
    devices = [device for device in use_gpu_device for _ in range(n_process_in_each_gpu)]    
    print("devices: ",devices)

    if not os.path.exists(results_dir):
        os.mkdir(results_dir)
    results_save_dir = results_dir + 'generate_01gamma_'
    meta_data_organ = None

    idxs = np.array_split(repeated_combinations, n_process)
    idxs = [list(idx) for idx in idxs]


    gammas = pack_arg(gamma,n_process)
    force_meta = pack_arg(True, n_process) 

    meta_terms = pack_arg(
         "FulLength"
        ,n_process)

    meta_organs = pack_arg(meta_data_organ,n_process)

    ckp_paths = pack_arg(ckp_path,n_process)
    results_save_dirs = [results_save_dir+str(process_id)+'results.pt' for _,process_id in zip(devices,process_idx)]

    args = [(idx, device, gamma, meta_organ, ckp_path, save_path, meta_term, force) \
            for idx, device, gamma, meta_organ,ckp_path,save_path,meta_term, force in \
            zip(idxs, devices,gammas, meta_organs, ckp_paths,results_save_dirs,meta_terms, force_meta)]
    with Pool(n_process) as pool:
        pool.map(generate_denovo.generate, args)

    end = time.time()
    print("Generation Finished, 总共用时{}秒".format((end - start)))